In [1]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt



In [2]:
# Generate synthetic sine wave data
def generate_sine_wave_data(num_points, amplitude=1.0, frequency=0.1, phase=0.0, noise_stddev=0.1):
    t = np.linspace(0, 10, num_points)
    y = amplitude * np.sin(2 * np.pi * frequency * t + phase)
    y += np.random.normal(0, noise_stddev, num_points)
    return t, y

# Create a dataset
num_points = 1000
t, y = generate_sine_wave_data(num_points)

# Convert data to PyTorch tensors
t = torch.tensor(t, dtype=torch.float32).view(-1, 1)
y = torch.tensor(y, dtype=torch.float32).view(-1, 1)


# Hyperparameters
input_size = 1
hidden_size = 64
num_layers = 2
output_size = 1
num_epochs = 100
learning_rate = 0.01



In [3]:
# Define an LSTM-based regression model
class LSTMRegressor(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMRegressor, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])  # Take the last time step's output
        return out


In [4]:
# Initialize the model and loss function
model = LSTMRegressor(input_size, hidden_size, num_layers, output_size)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(t)
    loss = criterion(outputs, y)

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluate the model
model.eval()
with torch.no_grad():
    predicted = model(t)



IndexError: too many indices for tensor of dimension 2

In [ ]:
# Plot the results
plt.figure(figsize=(10, 5))
plt.title('Sine Wave Regression using LSTM')
plt.scatter(t.numpy(), y.numpy(), label='True Data', s=10)
plt.plot(t.numpy(), predicted.numpy(), label='Predictions', linewidth=2, color='orange')
plt.legend()
plt.xlabel('Time')
plt.ylabel('Amplitude')
plt.show()